In [1]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from astral import LocationInfo
from astral.sun import sun
from datetime import datetime
import numpy as np

In [2]:
dataset_name = "load_forecasting.csv"
panama = LocationInfo("Panama City", "Panama", "America/Panama", 8.9824, -79.5199)

In [3]:
df = pd.read_csv(f"../datasets/{dataset_name}")
df.head()

,datetime,nat_demand,T2M_toc,QV2M_toc,TQL_toc,W2M_toc,T2M_san,QV2M_san,TQL_san,W2M_san,T2M_dav,QV2M_dav,TQL_dav,W2M_dav,Holiday_ID,holiday,school
0,2015-01-03 01:00:00,970.3450,25.865259,0.018576,0.016174,21.850546,23.482446,0.017272,0.001855,10.328949,22.662134,0.016562,0.096100,5.364148,0,0,0
1,2015-01-03 02:00:00,912.1755,25.899255,0.018653,0.016418,22.166944,23.399255,0.017265,0.001327,10.681517,22.578943,0.016509,0.087646,5.572471,0,0,0
2,2015-01-03 03:00:00,900.2688,25.937280,0.018768,0.015480,22.454911,23.343530,0.017211,0.001428,10.874924,22.531030,0.016479,0.078735,5.871184,0,0,0
3,2015-01-03 04:00:00,889.9538,25.957544,0.018890,0.016273,22.110481,23.238794,0.017128,0.002599,10.518620,22.512231,0.016487,0.068390,5.883621,0,0,0
4,2015-01-03 05:00:00,893.6865,25.973840,0.018981,0.017281,21.186089,23.075403,0.017059,0.001729,9.733589,22.481653,0.016456,0.064362,5.611724,0,0,0


In [4]:
def calculate_sun_intensity(date_time:datetime, city:LocationInfo=panama):
    """
    Calculate the sun intensity for a given datetime.
    The sun intensity is calculated based on the position of the sun in Panama.
    The intensity is 1 at solar noon, 0 before sunrise and after sunset, and 
    linearly interpolated between these times.
    Parameters:
    date_time (datetime): The datetime for which to calculate the sun intensity.
    Returns:
    float: The sun intensity, where 1 represents peak sun (solar noon) and 0 
           represents darkness (before sunrise or after sunset).
    """
    
    # Calculate sun times for Panama
    s = sun(city.observer, date=date_time)

    # 1 for peak sun (solar noon), 0 for time after sunset / before sunrise when its dark, values in between for other times - linear interpolation
    if date_time < s['sunrise'] or date_time > s['sunset']:
        return 0.0

    day_duration = (s['sunset'] - s['sunrise']).total_seconds()
    sunrise_seconds = (s['noon'] - s['sunrise']).total_seconds()
    sunset_seconds = day_duration - sunrise_seconds

    if date_time < s['noon']:
        return (date_time - s['sunrise']).total_seconds() / sunrise_seconds
    else:
        return (s['sunset'] - date_time).total_seconds() / sunset_seconds

In [5]:
def minmax_scale(df:pd.DataFrame, cols_to_avoid:list[str]=None):
    
    """
    Scales the columns of a DataFrame using Min-Max scaling.
    Parameters:
    df (pd.DataFrame): The input DataFrame with at least two columns, where the first two columns are 'datetime' and 'nat_demand'.
    Returns:
    pd.DataFrame: A new DataFrame with the numerical columns scaled between 0 and 1, while keeping the 'datetime' and 'nat_demand' columns unchanged.
    """

    df_initial = df.copy()
    scaler = MinMaxScaler()

    if cols_to_avoid:
        df_dropped = df_initial.drop(cols_to_avoid, axis=1)
        cols_after_drop = df_dropped.columns
    
    # do it for all columns except datetime and nat_demand
    df_scaled = pd.DataFrame(scaler.fit_transform(df_dropped), columns=cols_after_drop)
    # add datetime and nat_demand columns
    df_scaled = pd.concat([df_initial[cols_to_avoid], df_scaled], axis=1)

    return df_scaled

In [6]:
def calculate_month_progress(date_time:datetime):
    """
    Calculate progress through the month (0-1)
    Parameters:
    date_time (datetime): The datetime for which to calculate the progress through the month.
    Returns:
    float: The progress through the month, where 0 represents the first day of the month and 1 represents the last day of the month.
    """
    
    day, month, year = date_time.day, date_time.month, date_time.year
    days_in_month = pd.to_datetime(f'{year}-{month}-01').days_in_month
    
    return (day - 1) / days_in_month

In [7]:
def calculate_season(date:any, city:LocationInfo=panama):
    """
    Calculate the season progress based on the sun's position in Panama City.
    Returns a float value denoting progress within the season.
    For example, 1.1 denotes 10% into summer, 0.4 denotes 40% into spring, and so on.
    """
    # Calculate the sun's position for the given date and the solstices/equinoxes
    sun_info = sun(city.observer, date=date)
    spring_equinox = sun(city.observer, date=datetime(date.year, 3, 20, tzinfo=date.tzinfo))
    summer_solstice = sun(city.observer, date=datetime(date.year, 6, 21, tzinfo=date.tzinfo))
    fall_equinox = sun(city.observer, date=datetime(date.year, 9, 22, tzinfo=date.tzinfo))
    winter_solstice = sun(city.observer, date=datetime(date.year, 12, 21, tzinfo=date.tzinfo))

    # Determine the season based on the date
    if spring_equinox['noon'] <= sun_info['noon'] < summer_solstice['noon']:
        season = 0  # Spring
        start_date = spring_equinox['noon']
        end_date = summer_solstice['noon']
    elif summer_solstice['noon'] <= sun_info['noon'] < fall_equinox['noon']:
        season = 1  # Summer
        start_date = summer_solstice['noon']
        end_date = fall_equinox['noon']
    elif fall_equinox['noon'] <= sun_info['noon'] < winter_solstice['noon']:
        season = 2  # Fall
        start_date = fall_equinox['noon']
        end_date = winter_solstice['noon']
    else:
        season = 3  # Winter
        if sun_info['noon'] < spring_equinox['noon']:
            start_date = datetime(date.year - 1, 12, 21, tzinfo=date.tzinfo)
            end_date = spring_equinox['noon']
        else:
            start_date = winter_solstice['noon']
            end_date = datetime(date.year + 1, 3, 20, tzinfo=date.tzinfo)

    # Calculate the progress within the season
    total_days = (end_date - start_date).days
    days_passed = (sun_info['noon'] - start_date).days
    season_progress = days_passed / total_days

    return season + season_progress

In [8]:
def normalize_columns(df:pd.DataFrame, cols_to_avoid:list[str]=['datetime', 'nat_demand']):
    """
    Normalize the columns of a DataFrame.
    Parameters:
    df (pd.DataFrame): The input DataFrame with at least two columns, where the first two columns are 'datetime' and 'nat_demand'.
    Returns:
    pd.DataFrame: A new DataFrame with the numerical columns normalized, while keeping the 'datetime' and 'nat_demand' columns unchanged.
    """
    
    df_initial = df.copy()

    if cols_to_avoid:
        df_dropped = df_initial.drop(cols_to_avoid, axis=1)
    else:
        df_dropped = df_initial

    # Normalize the columns
    df_normalized = (df_dropped - df_dropped.min()) / (df_dropped.max() - df_dropped.min())
    
    if cols_to_avoid:
        df_normalized = pd.concat([df_initial[cols_to_avoid], df_normalized], axis=1)

    return df_normalized

In [9]:
def add_time_features(df:pd.DataFrame, date_column:str='datetime'):
    """
    Adds various time-based features to a DataFrame.
    Parameters:
    df (pd.DataFrame): The input DataFrame containing the data.
    date_column (str): The name of the column containing datetime information. Default is 'datetime'.
    Returns:
    pd.DataFrame: A DataFrame with additional time-based features.
    The function performs the following operations:
    - Applies Min-Max scaling and normalization to the DataFrame, excluding specified columns.
    - Converts the date_column to datetime and sets the timezone to 'America/Panama'.
    - Adds a 'month_day_progress' column representing the progress of the day in the month.
    - Adds a 'season' column representing the season of the date.
    - Adds a 'sun_intensity' column representing the sun intensity based on the date.
    - Adds sine and cosine encoding for the hour of the day.
    - Adds sine and cosine encoding for the progress of the day in the year.
    - Converts the datetime column to timezone-naive.
    Note:
    - The function assumes the existence of helper functions: `minmax_scale`, `normalize_columns`, 
        `calculate_month_progress`, `calculate_season`, and `calculate_sun_intensity`.
    - The columns 'nat_demand', 'Holiday_ID', and 'holiday' are excluded from scaling and normalization.
    """
    
    df = df.copy()
    cols_to_avoid = [date_column, 'nat_demand', 'Holiday_ID', 'holiday']

    df = minmax_scale(df, cols_to_avoid=cols_to_avoid) # apply minmax scaling
    df = normalize_columns(df, cols_to_avoid=cols_to_avoid) # apply normalization
    
    df[date_column] = pd.to_datetime(df[date_column])
    df[date_column] = df[date_column].dt.tz_localize('America/Panama') # set timezone to Panama
    
    df['month_day_progress'] = df[date_column].apply(
        lambda x: calculate_month_progress(x)
    )
    
    df['season'] = df[date_column].apply(
        lambda x: calculate_season(x)
    )

    df['sun_intensity'] = df[date_column].apply(
        lambda x: calculate_sun_intensity(x)
    )
    
    # Add position encoding for hour of the day
    df['hour'] = df[date_column].dt.hour
    df['hour_sin'] = np.sin(2 * np.pi * df['hour'] / 24)
    df['hour_cos'] = np.cos(2 * np.pi * df['hour'] / 24)
    
    # Add position encoding for day of the year
    df['year_progress'] = df[date_column].dt.dayofyear / 365
    df['year_progress_sin'] = np.sin(2 * np.pi * df['year_progress'])
    df['year_progress_cos'] = np.cos(2 * np.pi * df['year_progress'])
    
    # Convert datetime column to timezone-naive
    df[date_column] = df[date_column].dt.tz_convert(None)
    
    return df

In [10]:
df_final = add_time_features(df, date_column='datetime')

In [11]:
df_final.head()

,datetime,nat_demand,Holiday_ID,holiday,T2M_toc,QV2M_toc,TQL_toc,W2M_toc,T2M_san,QV2M_san,...,school,month_day_progress,season,sun_intensity,hour,hour_sin,hour_cos,year_progress,year_progress_sin,year_progress_cos
0,2015-01-03 06:00:00,970.3450,0,0,0.240921,0.613218,0.031030,0.556888,0.192620,0.589439,...,0.0,0.064516,3.146067,0.0,1,0.258819,0.965926,0.008219,0.05162,0.998667
1,2015-01-03 07:00:00,912.1755,0,0,0.243734,0.620449,0.031499,0.564955,0.188309,0.588850,...,0.0,0.064516,3.146067,0.0,2,0.500000,0.866025,0.008219,0.05162,0.998667
2,2015-01-03 08:00:00,900.2688,0,0,0.246880,0.631220,0.029698,0.572297,0.185422,0.584380,...,0.0,0.064516,3.146067,0.0,3,0.707107,0.707107,0.008219,0.05162,0.998667
3,2015-01-03 09:00:00,889.9538,0,0,0.248557,0.642709,0.031221,0.563516,0.179994,0.577349,...,0.0,0.064516,3.146067,0.0,4,0.866025,0.500000,0.008219,0.05162,0.998667
4,2015-01-03 10:00:00,893.6865,0,0,0.249905,0.651273,0.033153,0.539947,0.171528,0.571549,...,0.0,0.064516,3.146067,0.0,5,0.965926,0.258819,0.008219,0.05162,0.998667


In [12]:
df_final.to_csv(f"../output/{dataset_name.split('.')[0]}_new_cols.csv", index=False)